# Source Separation Using Classifier Transfer Learning

In [22]:
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Activation

## UNet Spectrogram Masker

In [26]:
class UNet:
    def __init__(self, input_shape, num_filters):
        self.input_shape = input_shape
        self.num_filters = num_filters
        self.build()

    def conv_block(self, input, num_filters):
        x = Conv2D(num_filters, 3, padding="same", activation="relu")(input)
        x = BatchNormalization()(x)
        x = Conv2D(num_filters, 3, padding="same", activation="relu")(x)
        x = BatchNormalization()(x)
        return x

    def encoder_block(self, input, num_filters):
        x = self.conv_block(input, num_filters)
        p = MaxPooling2D((2, 2))(x)
        return x, p

    def decoder_block(self, input, skip_features, num_filters):
        x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
        x = concatenate([x, skip_features], axis=3)
        x = self.conv_block(x, num_filters)
        return x

    def build(self):
        # Input
        inputs = Input(shape=self.input_shape)

        # Encoder
        e1, p1 = self.encoder_block(inputs, self.num_filters)
        e2, p2 = self.encoder_block(p1, self.num_filters*2)
        e3, p3 = self.encoder_block(p2, self.num_filters*4)
        e4, p4 = self.encoder_block(p3, self.num_filters*8)

        # Bridge
        b1 = self.conv_block(p4, self.num_filters*16)

        # Decoder
        d1 = self.decoder_block(b1, e4, self.num_filters*8)
        d2 = self.decoder_block(d1, e3, self.num_filters*4)
        d3 = self.decoder_block(d2, e2, self.num_filters*2)
        d4 = self.decoder_block(d3, e1, self.num_filters)

        # Output
        outputs = Conv2D(1, (1, 1), activation='sigmoid')(d4)

        # Model
        self.model = Model(inputs, outputs)

    def summary(self):
        self.model.summary()

    def compile(self, optimizer, loss, metrics=None):
        self.model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


In [27]:
model = UNet((512, 128, 1), 32)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 512, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 512, 128, 32  320         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_18 (BatchN  (None, 512, 128, 32  128        ['conv2d_20[0][0]']              
 ormalization)                  )                                                           